In [ ]:
from __future__ import print_function
import boto3
import logging
import os
import sys
import uuid
import pymysql
from datetime import datetime

logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = pymysql.connect(host='analyticszone.c5w8qvjfe5q6.ap-south-1.rds.amazonaws.com',
                             user='mydatalake',
                             password='ILHeIBwm7Rd5thpqw75t',
                             db='finaldata',
                             )
# 	conn = pymysql.connect(rds_host, user=name, passwd=password, db=db_name, connect_timeout=5)
except Exception as e:
	logger.error("ERROR: Unexpected error: Could not connect to MySql instance.")
	logger.error(e)
	sys.exit()
	
logger.info("SUCCESS: Connection to RDS mysql instance succeeded")

s3_client = boto3.client('s3')

def lambda_handler(event, context):
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8').split('\n')
    data.pop(0)
    data.pop(-1)
    print("Bucke:", bucket, key)
    print(data)
    with conn.cursor() as cur:
        if key.split('/')[-1] == 'ilearn.csv':
            for row in data:
                mail = row.split(',')[2]
                print(mail)
                cur.execute("select masterkey from KPIUser_List where email = '%s'" %(mail))
                masterkey = cur.fetchone()
                mas = masterkey[0]
                print("master: ",mas)
                fname = row.split(',')[0]
                lname = row.split(',')[1]
                course = row.split(',')[3]
                dur = row.split(',')[5]
                dur=dur.split("\r")[0]
                dur=int(dur)
                ins_date = datetime.now()
                ins_date = ins_date.strftime('%Y-%m-%d')
                query=(mas, fname, lname, mail,course, dur,ins_date)
                print(ins_date)
                logger.info(query)
                try:
                    cur.execute("INSERT INTO ilearn(masterkey, first_name,last_name, email, course, duration,ins_date) VALUES(%s, %s,%s, %s, %s, %s,%s)", (mas, fname, lname, mail,course, dur,ins_date))
                except Exception as e:
	                logger.error(e)
	                
    conn.commit()
    conn.close()

    return 'File loaded into RDS:' + str(key)
